In [1]:
from importlib import reload 
import lib.interpolate as functions
reload(functions) 
from lib.interpolate import *
import os
from more_itertools import pairwise
from collections import defaultdict

In [2]:
cwd = os.getcwd()
path = cwd+'/example/'

# Initialize interpolator

wav_ref, _ = np.loadtxt(cwd+'/database/lte050-4.5-0.0a+0.0.BT-NextGen.7.dat.txt', unpack=True)
interpolator = SpectrumInterpolator(wav_ref=wav_ref)

# Prepare inputs
targets = pd.read_csv('stars.csv') # List of objects to interpolate
target = targets.loc[targets['star'] == 'HAT-P-3 ']
spectra = pd.read_csv(path+'hat-p-3_data.csv')  # Your spectra data
interpolate_flags = pd.read_csv(path+'hat-p-3_interpolate.csv')

Initializing class...



In [3]:
target

,star,teff,logg,feh
4,HAT-P-3,5180,4.56,0.27


In [4]:
interpolate_flags

,teff,logg,feh
0,True,True,True


In [5]:
spectra

,teff,logg,feh,alpha,path
0,5100,4.5,0.5,0,/database/lte051-4.5+0.5a+0.0.BT-NextGen.7.dat...
1,5100,4.5,0.0,0,/database/lte051-4.5-0.0a+0.0.BT-NextGen.7.dat...
2,5100,5.0,0.5,0,/database/lte051-5.0+0.5a+0.0.BT-NextGen.7.dat...
3,5100,5.0,0.0,0,/database/lte051-5.0-0.0a+0.0.BT-NextGen.7.dat...
4,5200,4.5,0.5,0,/database/lte052-4.5+0.5a+0.0.BT-NextGen.7.dat...
5,5200,4.5,0.0,0,/database/lte052-4.5-0.0a+0.0.BT-NextGen.7.dat...
6,5200,5.0,0.5,0,/database/lte052-5.0+0.5a+0.0.BT-NextGen.7.dat...
7,5200,5.0,0.0,0,/database/lte052-5.0-0.0a+0.0.BT-NextGen.7.dat...


In [6]:
def sort_df(df, param, interpolate_flags):

    params = list(interpolate_flags.columns)

    results = []
    
    grouped = df.groupby([p for p in params if p != param])
    
    for _, group in grouped:
        min_row = group.loc[group[param].idxmin()]
        max_row = group.loc[group[param].idxmax()]
        results.extend([min_row, max_row])
    
    return pd.DataFrame(results) if results else pd.DataFrame()

In [7]:
test = sort_df(spectra, 'teff', interpolate_flags) 
test

,teff,logg,feh,alpha,path
1,5100,4.5,0.0,0,/database/lte051-4.5-0.0a+0.0.BT-NextGen.7.dat...
5,5200,4.5,0.0,0,/database/lte052-4.5-0.0a+0.0.BT-NextGen.7.dat...
0,5100,4.5,0.5,0,/database/lte051-4.5+0.5a+0.0.BT-NextGen.7.dat...
4,5200,4.5,0.5,0,/database/lte052-4.5+0.5a+0.0.BT-NextGen.7.dat...
3,5100,5.0,0.0,0,/database/lte051-5.0-0.0a+0.0.BT-NextGen.7.dat...
7,5200,5.0,0.0,0,/database/lte052-5.0-0.0a+0.0.BT-NextGen.7.dat...
2,5100,5.0,0.5,0,/database/lte051-5.0+0.5a+0.0.BT-NextGen.7.dat...
6,5200,5.0,0.5,0,/database/lte052-5.0+0.5a+0.0.BT-NextGen.7.dat...


In [8]:
def load_and_interpolate_spectrum(row):
        """
        Load spectrum from file and interpolate to wavelength resolution of reference.
        """

        wav, spec = np.loadtxt(cwd + row['path'], unpack=True)
        return np.interp(wav_ref, wav, spec)

In [9]:
params = {
        'teff': target['teff'].item(),
        'logg': target['logg'].item(),
        'feh': target['feh'].item()
        }

In [10]:
delta_params = {
        'teff': 100,  # [K]
        'logg': 0.5,  # [dex]
        'feh': 0.5    # [dex]
        }

In [11]:
def combine_dicts(dict_list):

    combined = defaultdict(list)
    for d in dict_list:
        for key, value in d.items():
            combined[key].append(value)

    return pd.DataFrame(dict(combined))

In [13]:
def interp_param(df, param):

    interp_steps = [] # This will storage each dictionary with updated parameters and the corresponding interpolated flux

    for i in tqdm(range(0, len(test), 2), desc='Interpolating '+str(param)):
        row1 = test.iloc[i, :].to_dict()  # First row of the pair
        row2 = test.iloc[i + 1, :].to_dict() if i + 1 < len(test) else None  # Second row (if exists)
        
        if row2 is None:
            pass

        try:
            spec1 = load_and_interpolate_spectrum(row1) # This will work only for the first parameter to be interpolated
            spec2 = load_and_interpolate_spectrum(row2)

        except:
            spec1 = row1['flux'] # Then for the rest of the parameters this should work
            spec2 = row2['flux']

        factor = params[param] - row1[param] # Desidered value - minimum parameter value

        interp_flux = interpolator.interp_partial(
            spec1,
            spec2,
            factor,
            delta_params[param]
        )

        dic = {'teff': params[param], # Fix later!
            'logg': row1['logg'],
            'feh': row1['feh'],
            'flux':interp_flux}

        interp_steps.append(dic)

    return interp_steps


In [15]:
interp_steps = interp_param(test, 'teff')

Interpolating teff: 100%|██████████| 4/4 [00:46<00:00, 11.60s/it]


In [16]:
df = combine_dicts(interp_steps)
df

,teff,logg,feh,flux
0,5180,4.5,0.0,"[3.7089199999999995e-99, 3.7089199999999995e-9..."
1,5180,4.5,0.5,"[3.709878000000001e-99, 3.709878000000001e-99,..."
2,5180,5.0,0.0,"[1.175568e-98, 1.175568e-98, 1.175568e-98, 1.1..."
3,5180,5.0,0.5,"[1.175302e-98, 1.175302e-98, 1.175302e-98, 1.1..."


In [17]:
sort_df(df, 'logg', interpolate_flags)

,teff,logg,feh,flux
0,5180,4.5,0.0,"[3.7089199999999995e-99, 3.7089199999999995e-9..."
2,5180,5.0,0.0,"[1.175568e-98, 1.175568e-98, 1.175568e-98, 1.1..."
1,5180,4.5,0.5,"[3.709878000000001e-99, 3.709878000000001e-99,..."
3,5180,5.0,0.5,"[1.175302e-98, 1.175302e-98, 1.175302e-98, 1.1..."
